In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
df_sampled = pd.read_csv('sampled_news.csv')

In [3]:
df_sampled.shape

(2000, 4)

In [4]:
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2000 non-null   int64 
 1   title       2000 non-null   object
 2   text        2000 non-null   object
 3   label       2000 non-null   object
dtypes: int64(1), object(3)
memory usage: 62.6+ KB


In [5]:
df_sampled['label'].value_counts(normalize=True) * 100

label
FAKE    51.15
REAL    48.85
Name: proportion, dtype: float64

In [6]:
df_sampled

,Unnamed: 0,title,text,label
0,9957,"American Dream, Revisited",Will Trump pull a Brexit times ten? What would...,FAKE
1,7596,Clintons Are Under Multiple FBI Investigations...,Clintons Are Under Multiple FBI Investigations...,FAKE
2,8905,The FBI Can’t Actually Investigate a Candidate...,Dispatches from Eric Zuesse This piece is cros...,FAKE
3,8752,Confirmed: Public overwhelmingly (10-to-1) say...,Print \n[Ed. – Every now and then the facade c...,FAKE
4,7804,Nanny In Jail After Force Feeding Baby To Death,Nanny In Jail After Force Feeding Baby To Deat...,FAKE
...,...,...,...,...
1995,7805,Saudis Foil ISIS Terror Attacks on Packed Stadium,Saudi Arabia says it has thwarted two ISIS ter...,FAKE
1996,10287,Police Department Is Being Evicted – Lafayette...,\nAn East Bay police department is now looking...,FAKE
1997,1176,"Trump, Sanders Crush the Competition in New Ha...",Democrat Bernie Sanders and Republican Donald ...,REAL
1998,5951,The words Extraordinary Claims needs to be ban...,The words Extraordinary Claims needs to be ban...,FAKE


In [7]:
if 'Unnamed: 0' in df_sampled.columns:
	df_sampled.drop(columns=['Unnamed: 0'], inplace=True)
if 'title' in df_sampled.columns:
	df_sampled.drop(columns=['title'], inplace=True)
df_sampled.head()

,text,label
0,Will Trump pull a Brexit times ten? What would...,FAKE
1,Clintons Are Under Multiple FBI Investigations...,FAKE
2,Dispatches from Eric Zuesse This piece is cros...,FAKE
3,Print \n[Ed. – Every now and then the facade c...,FAKE
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE


In [8]:
def clean_twitter_text(text):
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Menghapus mention
    text = re.sub(r'#\w+', '', text)  # Menghapus hashtag
    text = re.sub(r'RT[\s]+', '', text)  # Menghapus retweet
    text = re.sub(r'https?://\S+', '', text)  # Menghapus URL
    text = re.sub(r'\bfr\b', '', text, flags=re.IGNORECASE)  # Menghapus kata "fr" (case-insensitive)
    
    text = re.sub(r'[^A-Za-z0-9 ]', '', text)  # Menghapus karakter non-alphanumeric
    text = re.sub(r'\s+', ' ', text).strip()  # Menghapus spasi berlebih

    return text

df_sampled['text'] = df_sampled['text'].apply(clean_twitter_text)

In [9]:
df_sampled.head()

,text,label
0,Will Trump pull a Brexit times ten What would ...,FAKE
1,Clintons Are Under Multiple FBI Investigations...,FAKE
2,Dispatches from Eric Zuesse This piece is cros...,FAKE
3,Print Ed Every now and then the facade cracks ...,FAKE
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE


In [10]:
# Fungsi normalisasi untuk menghapus "rt" dari teks
def normalisasi(str_text):
    if isinstance(str_text, str):  # Mengecek apakah input adalah string
        # Menghapus semua kemunculan "rt, fr" di dalam string
        str_text = str_text.replace('rt ', '')
        str_text = str_text.replace('fr ', '')
        return str_text
    return str_text

# Menerapkan fungsi normalisasi pada kolom 'full_text'
df_sampled['text'] = df_sampled['text'].apply(normalisasi)

# Menampilkan hasil
print(df_sampled)

                                                   text label
0     Will Trump pull a Brexit times ten What would ...  FAKE
1     Clintons Are Under Multiple FBI Investigations...  FAKE
2     Dispatches from Eric Zuesse This piece is cros...  FAKE
3     Print Ed Every now and then the facade cracks ...  FAKE
4     Nanny In Jail After Force Feeding Baby To Deat...  FAKE
...                                                 ...   ...
1995  Saudi Arabia says it has thwarted two ISIS ter...  FAKE
1996  An East Bay police department is now looking f...  FAKE
1997  Democrat Bernie Sanders and Republican Donald ...  REAL
1998  The words Extraordinary Claims needs to be ban...  FAKE
1999  Home World Britain No Longer a Sovereign Democ...  FAKE

[2000 rows x 2 columns]


In [11]:
# Mengecek duplikat baris di dataset
duplikat = df_sampled.duplicated(subset='text')
print(duplikat.sum())  # Menghitung jumlah duplikat

# Menampilkan jumlah baris yang duplikat
print(f"Jumlah baris duplikat: {duplikat.sum()}")

# Menampilkan baris-baris yang duplikat
duplikat_baris = df_sampled[df_sampled.duplicated(subset='text')]
print("Baris duplikat:")
print(duplikat_baris)

61
Jumlah baris duplikat: 61
Baris duplikat:
                                                   text label
187                                                      FAKE
188                                                      FAKE
240   Killing Obama administration rules dismantling...  REAL
243   A verdict in 2017 could have sweeping conseque...  REAL
253                                                      FAKE
...                                                 ...   ...
1780  The president refuses to say hed hold to the t...  REAL
1816  Nation Puts 2016 Election Into Perspective By ...  FAKE
1829  The move would make it easier for the Trump ad...  REAL
1877  Nation Puts 2016 Election Into Perspective By ...  FAKE
1936  Killing Obama administration rules dismantling...  REAL

[61 rows x 2 columns]


In [12]:
df_sampled['text'] = df_sampled['text'].astype(str)
df_sampled['label'] = df_sampled['label'].astype(str)
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [13]:
df_sampled.drop_duplicates(subset="text", keep='first', inplace=True)  # Menghapus duplikat

# Mengecek duplikat baris di dataset
duplikat = df_sampled.duplicated(subset='text')
print(duplikat.sum())  # Menghitung jumlah duplikat

# Menampilkan jumlah baris yang duplikat
print(f"Jumlah baris duplikat: {duplikat.sum()}")

# Menampilkan baris-baris yang duplikat
duplikat_baris = df_sampled[df_sampled.duplicated(subset='text')]
print("Baris duplikat:")
print(duplikat_baris)

df_sampled.info()

0
Jumlah baris duplikat: 0
Baris duplikat:
Empty DataFrame
Columns: [text, label]
Index: []
<class 'pandas.core.frame.DataFrame'>
Index: 1939 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1939 non-null   object
 1   label   1939 non-null   object
dtypes: object(2)
memory usage: 45.4+ KB


In [14]:
# Mengecek jumlah nilai null di setiap kolom
null_counts = df_sampled.isnull().sum()
print(null_counts)

text     0
label    0
dtype: int64


In [15]:
def case_folding(text):
    if isinstance(text, str):
        lowercase_text = text.lower()
        return lowercase_text
    else:
        return text

df_sampled['case_folding'] = df_sampled['text'].apply(case_folding)

df_sampled.head()

,text,label,case_folding
0,Will Trump pull a Brexit times ten What would ...,FAKE,will trump pull a brexit times ten what would ...
1,Clintons Are Under Multiple FBI Investigations...,FAKE,clintons are under multiple fbi investigations...
2,Dispatches from Eric Zuesse This piece is cros...,FAKE,dispatches from eric zuesse this piece is cros...
3,Print Ed Every now and then the facade cracks ...,FAKE,print ed every now and then the facade cracks ...
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE,nanny in jail after force feeding baby to deat...


In [16]:
def tokenize(text):
    tokens = text.split()
    return tokens

df_sampled['tokenize'] = df_sampled['text'].apply(tokenize)
df_sampled.head()

,text,label,case_folding,tokenize
0,Will Trump pull a Brexit times ten What would ...,FAKE,will trump pull a brexit times ten what would ...,"[Will, Trump, pull, a, Brexit, times, ten, Wha..."
1,Clintons Are Under Multiple FBI Investigations...,FAKE,clintons are under multiple fbi investigations...,"[Clintons, Are, Under, Multiple, FBI, Investig..."
2,Dispatches from Eric Zuesse This piece is cros...,FAKE,dispatches from eric zuesse this piece is cros...,"[Dispatches, from, Eric, Zuesse, This, piece, ..."
3,Print Ed Every now and then the facade cracks ...,FAKE,print ed every now and then the facade cracks ...,"[Print, Ed, Every, now, and, then, the, facade..."
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE,nanny in jail after force feeding baby to deat...,"[Nanny, In, Jail, After, Force, Feeding, Baby,..."


In [17]:
from nltk.corpus import stopwords
import nltk

# Mengunduh stopwords
nltk.download('stopwords')

# Mengambil stopwords dari bahasa Inggris
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\YOGA
[nltk_data]     PRATAMA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df_sampled['stop_word'] = df_sampled['tokenize'].apply(lambda x: remove_stopwords(x))

df_sampled.head()

,text,label,case_folding,tokenize,stop_word
0,Will Trump pull a Brexit times ten What would ...,FAKE,will trump pull a brexit times ten what would ...,"[Will, Trump, pull, a, Brexit, times, ten, Wha...","[Will, Trump, pull, Brexit, times, ten, What, ..."
1,Clintons Are Under Multiple FBI Investigations...,FAKE,clintons are under multiple fbi investigations...,"[Clintons, Are, Under, Multiple, FBI, Investig...","[Clintons, Are, Under, Multiple, FBI, Investig..."
2,Dispatches from Eric Zuesse This piece is cros...,FAKE,dispatches from eric zuesse this piece is cros...,"[Dispatches, from, Eric, Zuesse, This, piece, ...","[Dispatches, Eric, Zuesse, This, piece, crossp..."
3,Print Ed Every now and then the facade cracks ...,FAKE,print ed every now and then the facade cracks ...,"[Print, Ed, Every, now, and, then, the, facade...","[Print, Ed, Every, facade, cracks, Somebody, a..."
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE,nanny in jail after force feeding baby to deat...,"[Nanny, In, Jail, After, Force, Feeding, Baby,...","[Nanny, In, Jail, After, Force, Feeding, Baby,..."


In [19]:
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\YOGA
[nltk_data]     PRATAMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
# Inisialisasi Porter Stemmer
stemmer = PorterStemmer()

def stemming(text):
    return [stemmer.stem(word) for word in text]

df_sampled['stemming'] = df_sampled['stop_word'].apply(lambda x: ' '.join(stemming(x)))
df_sampled.head()

,text,label,case_folding,tokenize,stop_word,stemming
0,Will Trump pull a Brexit times ten What would ...,FAKE,will trump pull a brexit times ten what would ...,"[Will, Trump, pull, a, Brexit, times, ten, Wha...","[Will, Trump, pull, Brexit, times, ten, What, ...",will trump pull brexit time ten what would tak...
1,Clintons Are Under Multiple FBI Investigations...,FAKE,clintons are under multiple fbi investigations...,"[Clintons, Are, Under, Multiple, FBI, Investig...","[Clintons, Are, Under, Multiple, FBI, Investig...",clinton are under multipl fbi investig agent a...
2,Dispatches from Eric Zuesse This piece is cros...,FAKE,dispatches from eric zuesse this piece is cros...,"[Dispatches, from, Eric, Zuesse, This, piece, ...","[Dispatches, Eric, Zuesse, This, piece, crossp...",dispatch eric zuess thi piec crosspost strateg...
3,Print Ed Every now and then the facade cracks ...,FAKE,print ed every now and then the facade cracks ...,"[Print, Ed, Every, now, and, then, the, facade...","[Print, Ed, Every, facade, cracks, Somebody, a...",print ed everi facad crack somebodi ask questi...
4,Nanny In Jail After Force Feeding Baby To Deat...,FAKE,nanny in jail after force feeding baby to deat...,"[Nanny, In, Jail, After, Force, Feeding, Baby,...","[Nanny, In, Jail, After, Force, Feeding, Baby,...",nanni in jail after forc feed babi to death 2 ...


In [21]:
df_sampled['label'].value_counts(normalize=True) * 100

label
FAKE    51.676122
REAL    48.323878
Name: proportion, dtype: float64

In [22]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# Mengubah teks menjadi representasi vektor menggunakan TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_sampled['stemming'])

# Menerapkan SVD
svd = TruncatedSVD(n_components=100)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X_lsa = lsa.fit_transform(X)

print(f"Shape of the original data: {X.shape}")
print(f"Shape of the data after SVD: {X_lsa.shape}")

Shape of the original data: (1939, 55655)
Shape of the data after SVD: (1939, 100)


In [23]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_sampled['stemming'])
X = tokenizer.texts_to_sequences(df_sampled['stemming'])

In [24]:
maxlen = 100
X = pad_sequences(X, padding='post', maxlen=maxlen)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, df_sampled['label'], test_size=0.2, random_state=42)
y_train = np.where(y_train == 'FAKE', 0, 1)
y_test = np.where(y_test == 'FAKE', 0, 1)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=maxlen))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

c:\Users\YOGA PRATAMA\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 174ms/step - accuracy: 0.5898 - loss: 0.6830 - val_accuracy: 0.7655 - val_loss: 0.6452
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - accuracy: 0.8362 - loss: 0.5142 - val_accuracy: 0.8376 - val_loss: 0.3917
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 130ms/step - accuracy: 0.9071 - loss: 0.3023 - val_accuracy: 0.7887 - val_loss: 0.7303
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.9015 - loss: 0.3185 - val_accuracy: 0.8299 - val_loss: 0.4083
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 150ms/step - accuracy: 0.9503 - loss: 0.1875 - val_accuracy: 0.8170 - val_loss: 0.5623


In [28]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)

13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
Accuracy: 81.70%
Confusion Matrix:
[[178  23]
 [ 48 139]]
